In [0]:
#!import "Storage"

In [0]:
public interface BasicCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<BasicCashflowDefinition> cd => GetStorage().GetBasicCashflowDefinitions(Identity);

    public RawVariable[] rv => cd.Select(x => new RawVariable() {
        Values = GenerateBasicCashflow(x),
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();
    

}

In [0]:
public interface BoundedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<BoundedCashflowDefinition> cd => GetStorage().GetBoundedCashflowDefinitions(Identity);

    public RawVariable[] rv => cd.Select(x => new RawVariable() {
        Values = GenerateBoundedCashflow(x),
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();
      
}

In [0]:
public interface ReferencedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public  IEnumerable<ReferencedCashflowDefinition> cd => GetStorage().GetReferencedCashflowDefinitions(Identity);

    public RawVariable[] rv => cd.Select(x => new RawVariable() {
        Values = GetScope<BasicCashflows>(Identity with {AmountType = x.ReferenceAmountType}).rv.First().Values,
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();

   
}